In [1]:
"""
Plot voltages from a model_run.py output file
"""
%matplotlib qt
import os
import matplotlib
matplotlib.use('Qt5Agg')
import pickle
import string
import numpy as np
import datetime
import textwrap
from cnmodel.util import sound
import matplotlib.pyplot as mpl
from pathlib import Path
import pylibrary.plotting.plothelpers as PH
from cnmodel.util import vector_strength
import json
from vcnmodel import cell_config

freq = 16000.
db = 30
SR = 'MS'
nreps = 50
fmod = 200.
dmod = 100

cconfig = cell_config.CellConfig()


def get_sims(cell, inp, mode='tonepip'):
    
    assert(mode in ['tonepip', 'SAM']) # Either 'SAM' or 'tonepip'
    scf = cconfig.get_soma_ratio(cell)
    basedir = '/Users/pbmanis/Desktop/Python/VCNModel'
    fnb = f'VCN_Cells/{cell:s}/Simulations/AN/AN_Result_{cell:s}_inp={inp:s}'  # or input = 'VCN_c02' etc
    titlestring = f"AN PSTH {freq/1000:.1f}kHz, {int(db):2d}dbSPL, {SR:s} fibers, {nreps:d} reps"

    if mode == 'tonepip':
        fnb += f'_XM13_nacncoop_II_soma={scf:.3f}_dend={scf:.3f}_ASA={scf:.3f}_delays_multisite_{nreps:03d}_tonepip_{db:03d}dB_{freq:.1f}_{SR:2s}.p'

    if mode == 'SAM':
        fnb += f'_XM13_nacncoop_II_soma={scf:.3f}_dend={scf:.3f}_ASA={scf:.3f}_delays_multisite_{nreps:03d}_SAM_{db:03d}dB_{freq:.1f}_{fmod:.1f}_{int(dmod):03d}_{SR:2s}.p'
        titlestring += f"SAM {int(fmod):d}/{dmod:d}%"

    return [Path(basedir, fnb)]
#     print('fnb: ', fnb)
#     matchstr = f"SAM_{db:03d}dB_{freq:.1f}_{fmod:.1f}_{int(dmod):03d})"
#     print('matchstr: ', matchstr)
#     print('match: ', fnb.find(matchstr))
#     fnsyn = Path(basedir, fnb[:(fnb.find(matchstr)+len(matchstr))])
#     print('fnsyn: ', fnsyn)
#     fns = fnsyn.parent.glob(str(fnsyn.name)+'*.p')
#     print('fns: ', list(fns))
#     fnsl0 = sorted(list(fns))
#     fnsl = []
#     print('fnsl0: ', fnsl0)
#     for i in range(len(fnsl0)):
#         if str(fnsl0[i]).find(f'_{nreps:03d}_SAM') > 0 and str(fnsl0[i]).find(f'_4000.0') > 0:
#             fnsl.append(fnsl0[i])
#     nax = len(fnsl)
#     for f in fnsl:
#         print('name: ', str(f.name))
#     return fnsl

# # fnb = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS.p'
# # fnbo = 'ForGeorge/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_001_tonepip_030dB_16000.0_HS_noPosixPath.p'
# # VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p

# fnb = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"

# fnb2 = "VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p"
# print(os.getcwd())

# if not Path(fnb).is_file():
#     print('not a file? ', fnb)
#     assert False

# if not Path(fnb2).is_file():
#     print('not a file? ', fnb2)
#     assert False

def get_one(fn):
    d = pickle.load(open(fn, 'rb'))
#     print(d.keys())
#     print('NTrials: ', len(d['trials']))
#     print(d['Params'].keys())
    return d

# d = get_one(fnb)
# # print(d['trials'][0]['inputSpikeTimes'])
# # print(d)

# for v in ['initIVStateFile', 'initANStateFile', 'simulationFilename', 'hocfile']:
#     d['Params'][v] = str(d['Params'][v])
# # print(d['Params'])
# # with(open(fnbo, 'wb')) as fh:
# #     pickle.dump(d, fh)



FileNotFoundError: [Errno 2] No such file or directory: 'wheres_my_data.toml'

In [2]:
def make_sound(stimInfo):
    if isinstance(stimInfo['pip_start'], list):
        pips = stimInfo['pip_start']
    else:
        pips = [stimInfo['pip_start']]
    if stimInfo['soundtype'] == 'tonepip':
        stim = sound.TonePip(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    elif stimInfo['soundtype'] == 'SAM':
        stim = sound.SAMTone(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], 
                          fmod=stimInfo['fmod'], dmod=stimInfo['dmod'],
                          pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    else:
        raise ValueError('StimInfo sound type %s not implemented' % stimInfo['soundtype'])

    stimWaveform = stim.generate()
    stimTimebase = stim.time
    return(stimTimebase, stimWaveform)

In [33]:
def get_cell_data(data, dataset, anf='all', fslonly=False):
    allst = []
    pipstart = data['Params']['pip_start'][0]
    
    if dataset == 'cell':
        for trial in data['trials']:
            # print (trial)
            td  = np.array(data['trials'][trial]['spikeTimes'])/1000.
            if fslonly:
                td = [td[td>pipstart][0]]
            allst.extend(td)
    elif dataset == 'AN':
        for trial in data['trials']:
            if anf == 'all':
#                 print('ALL: len an spike times for trial: ', trial, len(data['trials'][trial]['inputSpikeTimes']))
                for n in range(len(data['trials'][trial]['inputSpikeTimes'])):
                    td = np.array(data['trials'][trial]['inputSpikeTimes'][n])/1000.
                    if fslonly:
                        td = [td[td>pipstart][0]]
                    allst.extend(td)
            else:
                td = np.array(data['trials'][trial]['inputSpikeTimes'][anf])/1000.
                if fslonly:
                    td = [td[td>pipstart][0]]
                allst.extend(td)
                
    else:
        raise ValueError('dataset must be either "cell" or "AN"')
    return allst

In [45]:
 def computeVS(data, dataset='cell', anf='all', verbose=True):
    """
    data is the dictionary of data

    dataset is which data will be analyzed. 
    Set to 'cell' to analyze the cell's spike train
    Set to 'AN' to analyze the auditory nerve input


    anf selects which auditory nerve fiber to analyze
    """
        
    allst = get_cell_data(data, dataset, anf=anf)
    null_VS = {'r': 0., 'n': 0, }
    allst = np.array(allst)
    allst = np.sort(allst)
#     if dataset == 'AN':
#         print('allst2: ', allst)
    si = data['trials'][0]['stimInfo']
    if si['soundtype'] == 'SAM':  # calculate vs and plot histogram
        # print('allst: ', allst)
        phasewin = [data['Params']['pip_start'][0] + 0.25*data['Params']['pip_duration'], 
            data['Params']['pip_start'][0] + data['Params']['pip_duration']]
        # print (phasewin)
        spkin = allst[np.where(allst > phasewin[0])]
        spikesinwin = spkin[np.where(spkin <= phasewin[1])]

        # set freq for VS calculation
        if si['soundtype'] == 'tone':
            f0 = data['Params']['F0']
            if verbose:
                print ("Tone: f0=%.3f at %3.1f dbSPL, cell CF=%.3f" % 
                    (data['Params']['f0'], data['Params']['dB'], data['Params']['F0'] ))
        if si['soundtype'] == 'SAM':
            f0 = data['Params']['fmod']
            tstring = ("SAM Tone: f0=%.3f at %3.1f dbSPL, fMod=%3.1f  dMod=%5.2f, cell CF=%.3f" %
                 (data['Params']['F0'], data['Params']['dB'], data['Params']['fmod'], data['Params']['dmod'], data['Params']['F0']))
            if verbose:
                print(tstring)
            # P.figure_handle.suptitle(tstring + '\n' + self.fn[self.Params['patterns'][0]].replace('_', '\_'), fontsize=9)
       # print('spikes: ', spikesinwin)
        if verbose:
            print((' Sound type: ', si['soundtype']))
        if len(spikesinwin) < 10:
            vs = make_null_vs(len(spikesinwin))
            if verbose:
                print('AN Vector Strength: Insufficient spike count, n = %d' % (len(spikesinwin)))
        else:
            vs = vector_strength(spikesinwin*1000., f0)  # vs expects spikes in msec
            if verbose:
                print(f"AN Vector Strength [anf={str(anf):s}] at {f0:.1f} = {vs['r']:7.3f}, d={vs['d']:.2f} (us) Rayleigh: {vs['R']:7.3f}  p = {vs['p']:.3e}  n = {vs['n']:d}")
        return(vs)
    else:
        return make_null_vs(0)

In [46]:
def make_null_vs(n):
    null_vs = {'r': 0., 'n': n, 'R': 0., 'p': 1.0, 'ph': np.zeros(n), 'd': 0.}
    return null_vs

In [47]:
def compute_fsl(data):
    maxfsl = 0.010  # maximum time to be included in FSL distribution
    pan = []
    psc = []
    fslan = []
    fslc = []
    t0 = data['Params']['pip_start']
    for trial in data['trials']:
        latan = np.array(data['trials'][trial]['inputSpikeTimes'][0])/1000.-t0
        fslan.append(latan[latan > 0][0])
        pan.extend(latan)
        
        latc = np.array(data['trials'][trial]['spikeTimes'])/1000.-t0
        fslc.append(latc[latc > 0][0])
        psc.extend(latc)

    fslan = np.array(fslan)
    fslan = fslan[fslan < maxfsl]
    print(f"AN: N={len(fslan):d}  mean={1e3*np.mean(fslan):.4f} std={1e3*np.std(fslan):.4f}", end='')
    print(f"      median: {1e3*np.median(fslan):.4f}")

    fslc = np.array(fslc)
    fslc = fslc[fslc < maxfsl]
    print(f"Postsynaptic FSL: N={len(fslc):d}  mean={1e3*np.mean(fslc):.4f} std={1e3*np.std(fslc):.4f}", end='')
    print(f"      median: {1e3*np.median(fslc):.4f}")
    


In [54]:
def plot_SAM(P, ax, cell, inp, fnsl):
    # f, ax = mpl.subplots(3,1, figsize=(6,6))
    
#     P.figure_handle.suptitle(titlestring, fontsize=10)
    naxn = len(ax)
#     print('fnsl: ', fnsl)
    fn = fnsl
    d = get_one(fn)
    print('input file: ', fn)
    
    st = get_cell_data(d, 'cell')
    st = np.array(st).ravel()*1000. # convert to msec
    ax[0].hist(st, bins=np.arange(0, 250., 0.5))
    ax[0].set_ylim((0,40))
    ax[0].set_ylabel('spike count')

    PH.nice_plot(ax[0])
    vs = computeVS(d, dataset='cell')
    print('computed vs')
    compute_fsl(d)
    print('computed fsl')

    stimInfo = d['trials'][0]['stimInfo']
    f0 = stimInfo['F0']
    dbspl = stimInfo['dB']
    fmod = stimInfo['fmod']
    dmod = stimInfo['dmod']
    print('SAM: fmod, domd: ', fmod, dmod)

    s=f"Cell:{cell:s} Input: {inp:s}  VS={vs['r']:.3f}\nN spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
    s += f"\nF: {float(f0):.1f}Hz {int(dbspl):3d}dB Fmod={fmod:.1f}Hz %mod={int(dmod):3d}"
    filetime = fn.stat().st_mtime
    sfiletime = datetime.datetime.utcfromtimestamp(filetime).strftime('%Y-%m-%dT%H:%M:%S:%SZ')
    s += f"\nRun Date: {sfiletime:s}"
    wfn = textwrap.wrap(str(fn.name), width=60)
    for i, w in enumerate(wfn):
        if i == 0:
            s += f"\nFile: {w:s}"
        else:
            s += f"\n      {w:s}"
    ax[0].text(x=0.05, y=1.15,
                 s=s,
                 transform=ax[0].transAxes, fontsize=7)

    print('SAM: get cell data')
    anst = get_cell_data(d, 'AN')
#     print(len(anst))
#     for i in range(len(anst)):
#         print(len(anst[i]))
# #           print('anst: ', anst)


    anst = np.array(anst).ravel()*1000. # convert to msec
    ax[1].hist(anst, bins=np.arange(0, 250., 0.5))
    ax[1].set_ylim((0,100))
    ax[1].set_ylabel('AN spike count')
    PH.nice_plot(ax[1])

    vsin = computeVS(d, dataset='AN', anf=0)
#     print('vsin: ', vsin)
    an_vstxt = f"VS={vsin['r']:.3f} N={vsin['n']:d}"
    ax[1].text(x=0.5, y=0.95,
                 s=an_vstxt,
                 transform=ax[1].transAxes, horizontalalignment='center', fontsize=7)

    print('SAM: make sound')


    t, w = make_sound(d['trials'][0]['stimInfo'])
    ax[2].plot(t*1000., w, linewidth=0.5)
    ax[2].set_xlabel('T (ms)')

    # get VS from each fiber
    nAN = len(d['trials'][0]['inputSpikeTimes'])
    for i in range(nAN):
        vs = computeVS(d, dataset='AN', anf=i, verbose=False)
        s=f"Cell:{cell:s} Input: {inp:s} AN#: {i:2d} VS={vs['r']:.3f}  N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
#         s += f"\nF: {float(f0):.1f}Hz {int(dbspl):3d}dB Fmod={fmod:.1f}Hz %mod={int(dmod):3d}"
        print(s)
        
#     # get AN data from one fiber:
#     ansp = []
#     for trial in d['trials']:
#                 # print (trial)
#         ansp.extend(np.array(d['trials'][trial]['inputSpikeTimes'][0])/1000.)
# #         print(len(d['trials'][trial]['inputSpikeTimes'][0]))
#     ax[1].hist(np.array(ansp)*1000., bins=np.arange(0, 250., 0.5))
#     ax[1].set_ylabel('AN spike count')           
#     PH.nice_plot(ax[1])
    for i in range(0,naxn):
        ax[i].set_xlim(75, 225)
        ax[i].set_xlabel('T (ms)')
    
    #mpl.show()
#     P.figure_handle.tight_layout()

#     mpl.savefig(outname)
#     mpl.show()                    

In [55]:
def plot_multiple_runs(mode='PSTH', fslonly=False):
    print('Mode: ', mode)

    P = PH.regular_grid(3, len(cells), figsize=(12, 8),  order='columnsfirst',
                verticalspacing=0.1, horizontalspacing=0.05,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.15, 'bottommargin': 0.1},
                labelposition=(-0.15, 1.05), panel_labels=string.ascii_uppercase[0:3*len(cells)])
    ax = P.axarr.ravel()
    
    for i, cell in enumerate(cells):
        print('cell, inputs: ', cell, inps[i])
        fnsl = get_sims(cell, inps[i], mode=mode)
        ax0 = i*3
        if mode == 'SAM':
            plot_SAM(P, ax[ax0:ax0+3], cell, inps[i], fnsl[0])
        elif mode == 'PSTH':
            plot_PSTH(P, ax[ax0:ax0+3], cell, inps[i], fnsl[0], fslonly=fslonly)
    mpl.show()

In [56]:
def plot_single_cell(file=None, cell='', inp=''):
    P = PH.regular_grid(3, 1, figsize=(6, 5),  order='columnsfirst',
                verticalspacing=0.1, horizontalspacing=0.05,
                margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.25, 'bottommargin': 0.1},
                labelposition=(-0.15, 1.05), panel_labels=string.ascii_uppercase[0:3*len(cells)])
    ax = P.axarr.ravel()

    plot_SAM(P, ax, cell, inp, fnsl)
    mpl.show()
    

In [57]:
def plot_sims(cell, inp, fnsl):
    # PLOT FOR AN INPUT (one spike train)
    # f, ax = mpl.subplots(3,1, figsize=(8,6))
    P = PH.regular_grid(2, 1, figsize=(8,4),
                    verticalspacing=0.2, horizontalspacing=0.08,
                    margins={'leftmargin': 0.1, 'rightmargin': 0.1, 'topmargin': 0.1, 'bottommargin': 0.1})
    P.figure_handle.suptitle(titlestring, fontsize=10)
    ax = P.axarr.ravel()
    print('fnsl: ', fnsl)
    for iax, fn in enumerate(fnsl): #, fnb2]):
        d = get_one(fn)
#         print(d['trials'][0].keys())
        st = []
        for i in list(d['trials'].keys()):
            for ansp in d['trials'][i]['spikeTimes']:
#                 print('ansp: ', ansp)
                if isinstance(ansp, list):
                    st.extend(ansp)
                else:
                    st.append(ansp)
        st = np.array(st).ravel()
        ax[iax].hist(st, bins=np.arange(0, 250., 0.5))
        ax[iax].set_ylim((0,15))
#         PH.referenceline(ax[iax], 10)
        ax[iax].set_ylabel('spike count')

        PH.nice_plot(ax[iax])
        vs = computeVS(d)
        if iax == 0:
            ax[iax].text(x=0.05, y=0.85,
                     s=f"Cell:{cell:s} Input: {inp:s}  VS={vs['r']:.3f}, N spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}",
                     transform=ax[iax].transAxes, fontsize=8)
            
    t, w = make_sound(d['trials'][0]['stimInfo'])
    ax[1].plot(t*1000, w, linewidth=0.5)
    ax[1].set_xlabel('T (ms)')

    PH.nice_plot(ax[iax])
    for i in range(0,2):
        ax[i].set_xlim(75, 225)
    #mpl.show()
    P.figure_handle.tight_layout()



In [58]:
def plot_PSTH(P, ax, cell, inp, fnsl, fslonly=False):
    # f, ax = mpl.subplots(3,1, figsize=(6,6))
    
#     P.figure_handle.suptitle(titlestring, fontsize=10)
    naxn = len(ax)
#     print('fnsl: ', fnsl)
    if fslonly:
        tstart=95.0
        tend = 120.0
        binw = 0.25

    else:
        tstart = 75.
        tend = 225.
        binw = 0.5

    fn = fnsl
    d = get_one(fn)
    st = get_cell_data(d, 'cell', fslonly=fslonly)
    st = np.array(st).ravel()*1000. # convert to msec
    ax[0].hist(st, bins=np.arange(0, 250., binw))
    ax[0].set_ylim((0,20))
    ax[0].set_ylabel('spike count')
    stimInfo = d['trials'][0]['stimInfo']
    f0 = stimInfo['F0']
    dbspl = stimInfo['dB']
    s=f"Cell:{cell:s} Input: {inp:s}  "
#     "VS={vs['r']:.3f}\nN spikes={vs['n']:d} R={vs['R']:7.2f} p={vs['p']:.3e}"
    s += f"\nF: {float(f0):.1f}Hz {int(dbspl):3d}dB"
    print(s)

    PH.nice_plot(ax[0])

    compute_fsl(d)

    filetime = fn.stat().st_mtime
    sfiletime = datetime.datetime.utcfromtimestamp(filetime).strftime('%Y-%m-%dT%H:%M:%S:%SZ')
    s += f"\nRun Date: {sfiletime:s}"
    wfn = textwrap.wrap(str(fn.name), width=40)
    for i, w in enumerate(wfn):
        if i == 0:
            s += f"\nFile: {w:s}"
        else:
            s += f"\n      {w:s}"
    ax[0].text(x=0.05, y=1.15,
                 s=s,
                 transform=ax[0].transAxes, fontsize=7)

    anst = get_cell_data(d, 'AN', fslonly=fslonly)
    anst = np.array(anst).ravel()*1000. # convert to msec
    ax[1].hist(anst, bins=np.arange(0, 250., binw))
    ax[1].set_ylim((0,60))
    ax[1].set_ylabel('AN spike count')
    PH.nice_plot(ax[1])

    t, w = make_sound(d['trials'][0]['stimInfo'])
    ax[2].plot(t*1000., w, linewidth=0.5)
    ax[2].set_xlabel('T (ms)')

#     # get AN data from one fiber:
#     ansp = []
#     for trial in d['trials']:
#                 # print (trial)
#         ansp.extend(np.array(d['trials'][trial]['inputSpikeTimes'][0])/1000.)
# #         print(len(d['trials'][trial]['inputSpikeTimes'][0]))
#     ax[1].hist(np.array(ansp)*1000., bins=np.arange(0, 250., 0.5))
#     ax[1].set_ylabel('AN spike count')           
#     PH.nice_plot(ax[1])
    for i in range(0,naxn):
        ax[i].set_xlim(tstart, tend)
        ax[i].set_xlabel('T (ms)')
    
    #mpl.show()
#     P.figure_handle.tight_layout()

#     mpl.savefig(outname)
#     mpl.show()            

In [59]:
import string
inps =  ['self']*4 # 'VCN_c13'
cells = ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
plot_multiple_runs(mode='SAM', fslonly=False)

Mode:  SAM
cell, inputs:  VCN_c09 self
Cell: 09: Soma mesh area: 1340.36  Soma hoc area: 885.30        Soma Inflation ratio: 1.514
input file:  /Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self_XM13_nacncoop_II_soma=1.514_dend=1.514_ASA=1.514_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength [anf=all] at 200.0 =   0.853, d=0.00 (us) Rayleigh: 604.861  p = 7.872e-225  n = 709
computed vs
AN: N=50  mean=4.0650 std=1.4722      median: 3.5300
Postsynaptic FSL: N=49  mean=4.4740 std=0.3341      median: 4.4000
computed fsl
SAM: fmod, domd:  200.0 100.0
SAM: get cell data
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength [anf=0] at 200.0 =   0.646, d=0.00 (us) Rayleigh: 317.960  p = 5.745e-90  n = 492
SAM: make sound
Cell:VCN_c09 Input: self AN

In [1]:
import string
inps =  ['self']*4 # 'VCN_c13'
cells = ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
plot_multiple_runs(mode='SAM')

NameError: name 'plot_multiple_runs' is not defined

In [60]:
inps = ['VCN_c02', 'VCN_c05', 'VCN_c10', 'VCN_c13'] # ['self']*4 # 'VCN_c13'
cells = ['VCN_c09']*4 # ['VCN_c09', 'VCN_c11', 'VCN_c17', 'VCN_c18']
plot_multiple_runs(mode='SAM', fslonly=False)


Mode:  SAM
cell, inputs:  VCN_c09 VCN_c02
Cell: 09: Soma mesh area: 1340.36  Soma hoc area: 885.30        Soma Inflation ratio: 1.514
input file:  /Users/pbmanis/Desktop/Python/VCNModel/VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=VCN_c02_XM13_nacncoop_II_soma=1.514_dend=1.514_ASA=1.514_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength [anf=all] at 200.0 =   0.851, d=0.00 (us) Rayleigh: 580.931  p = 2.563e-215  n = 683
computed vs
AN: N=48  mean=3.8242 std=1.2874      median: 3.5350
Postsynaptic FSL: N=49  mean=4.6679 std=0.8198      median: 4.5250
computed fsl
SAM: fmod, domd:  200.0 100.0
SAM: get cell data
SAM Tone: f0=16000.000 at 30.0 dbSPL, fMod=200.0  dMod=100.00, cell CF=16000.000
(' Sound type: ', 'SAM')
AN Vector Strength [anf=0] at 200.0 =   0.654, d=0.00 (us) Rayleigh: 325.910  p = 2.347e-93  n = 498
SAM: make sound
Cell:VCN_c09 Input: V

In [ ]:
cell='VCN_c09'
inp = 'self'
fn = 'AN_Result_VCN_c09_inp=self_XM13nacncoop_II_soma=1.489_dend=1.236_delays_multisite_050_SAM_030dB_16000.0_200.0_100_MS.p'
basedir = '/Users/pbmanis/Desktop/Python/VCNModel'
fnb = f'VCN_Cells/{cell:s}/Simulations/AN/'
fnsl = Path(basedir, fnb, fn)
plot_single_cell(cell=cell, inp=inp, file=fnsl)
